'''
RAGAs needs: 
    Ground Truth: Actual Document --> URL connection to url in doc dataset
    Answer: RAG output
    Contexts: passage texts in passages
    Question: query and query id


Merge datasests on matching URLs, with columns:
    From msm_doc: url, title, headings, body
    From msm_QA:url, passage_txt, query, query_id

How to split the data:
    contexts: all passage_texts based on query_id


'''

In [ ]:

import itertools
import json
import pandas as pd
import gzip
import json
import io
import os

In [ ]:
# Opening JSON file
# qa_path = open('MSMARCO-Question-Answering/eval_v2.1_public.json')
qa_path = open('MSMARCO-Question-Answering/train_v2.1.json')
# qa_path = open('MSMARCO-Question-Answering/dev_v2.1.json')


# returns JSON object as 
# a dictionary
data_dict = json.load(qa_path)

In [ ]:
# data_dict is a nested dictionary
# here we create individial dictionaries for each column
passages = data_dict["passages"]
query = data_dict["query"]
query_id = data_dict["query_id"]
query_type = data_dict['query_type']
answers = data_dict["answers"]

#now we convert the seaparate dictionaries to a dataframe
rows = []
for outer_key, items in passages.items():
    for item in items:
        item['key'] = outer_key
        rows.append(item)
df = pd.DataFrame(rows)

df['query'] = df['key'].map(query)
df['query_id'] = df['key'].map(query_id)
df['query_type'] = df['key'].map(query_type)
df['answers'] = df['key'].map(answers)

#format the passages to get it ready for context
passages = []
for passage_txt in df['passage_text']:
    passages.append(passage_txt)
    # passages = " ".join([passages, str(passage)])
# print(passages.lstrip())
# print(passages[0:2])

df['contexts'] = passages
print(type(df['contexts']))

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns format
# print(len(df['key']))
print("df", len(df))

In [ ]:
#  Count the unique values in the 'category' column
# unique_category_count = df['key'].nunique()
# print(f"Number of unique categories: {unique_category_count}")

#now we read in the MSMarco documents dataset
docs_path = "./datasets/TREC_RAG_2024/msmarco_v2.1_doc/msmarco_v2.1_doc_00.json.gz"

#read in the .json.gz zip
#output is a list of strings with dictionary formating
data = []
gz = gzip.open(docs_path, 'rb')
f = io.BufferedReader(gz)

for line in f:
    line = line.decode('utf-8')
    # print("line", line)
    data.append(line)
gz.close()

# print(type(data))
length = len(data)
#193732 in the MSMarco_docs00
# print("number of documents in data:", length) 
# print(type(data))


In [ ]:
#now convert the strings to dictionaries to put into a dataframe
# Initialize an empty list to hold the dictionaries
dictionaries = []

# Iterate over the list of strings
for item_str in data:
    try:
        # Parse the string into a dictionary
        item_dict = json.loads(item_str)
        # Append the dictionary to the list
        dictionaries.append(item_dict)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

# Print the list of dictionaries
# for d in dictionaries:
#     print(type(d))

df_docs = pd.DataFrame(dictionaries)
print("df_docs",len(df_docs))

# see what this looks like
# print(df['passage_text'])


In [ ]:
###Now merge with QA on matching urls
# see if you need to remove the query_ids
# implement them into ragas

MSMarco_QA_Docs = pd.merge(df, df_docs, on = "url", how = "inner")
print(len(MSMarco_QA_Docs))
MSMarco_QA_Docs = MSMarco_QA_Docs.rename(columns = {'query':"question", "answers":"answer", "body":"ground_truth"})

#split the data into batches to run evaluaiton and metrics better
df_list = list()
for i in range(12):
    # temp_df = df.sample(1000)
    temp_df = MSMarco_QA_Docs.sample(1000)
    df_list.append(temp_df) # list of 12 df's 1000 long each
    
print("df_list:", len(df_list))
print("df_list:", df_list[0])

MSMarco_QA_Docs.to_csv('MSMarco_QA_Docs_sample.csv')
